# Unit 2 Lesson 7 - Joint 2D and AngryAvians
In this lesson, we'll learn about new Unity physics components called Joint2D and use everything we learned so far to create AngryAvians(Angry Birds clone).

---

# Learning Objectives
---

*   Students will learn about Joint2D
*   Students will apply previous lesson to create an Angry Birds clone

# Key Concepts
---

*   Joint2D
*   Inheritance

# Introduction
---

Another useful physics components that Unity provide are the varieties of Joint2D. Joint2D allows for GameObjects to attach together based on different joints behavior. While, there are a lot of Joint2D to explore, for this Angry Birds clone, we will only use HingeJoint2D. It is recommended to checkout the 2D Joints documentation to learn more about the components.

Unity Documentation: https://docs.unity3d.com/Manual/Joints2D.html

For this project, we will use bit and pieces of concepts that we learned so far from previous lessons to build out an Angry Birds clone, Angry Avians!

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_1.png" alt="Drawing"/>

All of the sprites used in this lesson will be provided.

# Concept 1: HingeJoint2D, SpringJoint2D

---
### **HingeJoint2D**

#### What is HingeJoint2D? 
From the Unity Documentation, HingeJoint2D allows GameObject to be attached to a point in space which it can rotate around. 

Documentation: https://docs.unity3d.com/Manual/class-HingeJoint2D.html

#### How to use HingeJoint2D?

To use HingeJoint2D, we need to have in mind what object we want to rotate around from a point. In this project, there are two instances of Joints
that were used. First, let's look at how to use hinge joint to create a beam

1. Create a game object using the beam sprite. Attach rigidbody2d and collider.
2. Attach HingeJoint2D. We can also change the anchor location where the beam is set to rotate around.

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_2.png" alt="Drawing"/>

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_1.gif" alt="Drawing"/>



## Walkthrough: Adding birds and SpringJoints
---

#### What is SpringJoint2D? 
SprintJoint2D allows for 2 gameobjects to attach to each other as if a spring is connecting them. This is good for anything that required stretchiness, like bird and slingshot. 
Documentation: https://docs.unity3d.com/Manual/class-SpringJoint2D.html


Let's now apply SpringJoint to our bird and use our mouse to drag the bird around.

1. Create the slingshot for bird anchor location. First, create the Slingshot game object with the following structure:

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_3.png" alt="Drawing"/>

The parent Slingshot game object, will have the front sprite of the slingshot, with a rigidbody2d set to static.

The backsprite will use the backsprite of the slingshot.

Anchor will be the center between slingshot and backsprite.

2. Next, let's set up the bird anchor onto the slingshot. First, create the game object with the bird sprite, and attach rigidbody, and collider.

3. Then, attach SpringJoint2D component to the bird. 

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_4.png" alt="Drawing"/>

We want "Connected Rigid Body" to be attach to the slingshot game object that we created previously.

Next, we want to move our bird closer to the anchor, where "distance" is close to 0

Lastly, we want to "Frequency" to be aroudn 1.5.

4. Let's create scripts to control move the bird, and shoot the bird from the slingshot. First, create a script called "Slingshot" and attach to our slingshot game object:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;
using UnityEngine.UIElements;

public class Slingshot : MonoBehaviour
{
    private int index = 0;
    [SerializeField]
    private BirdController[] birds;
    public BirdController currentBird;
    public BirdController releasedBird;

    [SerializeField] private float MAX_DISTANCE;

    [SerializeField]
    private Transform anchor;

    [SerializeField]
    private float releaseTime;

    private bool isMouseDown;
    public bool HoldBird { get; private set;  }
    
    private Rigidbody2D rb;
    private SpringJoint2D springJoint;

    // Start is called before the first frame update
    void Start()
    {
        currentBird = birds[index++];
        rb = currentBird.RB;
        springJoint = currentBird.SpringJoint;
    }

    // Update is called once per frame
    void Update()
    {

        if (Input.GetKeyDown(KeyCode.R))
        {
            SceneManager.LoadScene(SceneManager.GetActiveScene().name);
        }

        if (Input.GetMouseButtonDown((int)MouseButton.LeftMouse))
        {
            isMouseDown = true;
            releasedBird = null; // kinda jank // this stops camera to interp to release bird
        }

        if (Input.GetMouseButtonUp((int)MouseButton.LeftMouse))
        {
            rb.isKinematic = false;
            isMouseDown = false;

            // Release
            if (HoldBird)
            {
                StartCoroutine(Release(releaseTime));
            }
            HoldBird = false;
        }

        if (isMouseDown && HoldBird)
        {
            Vector2 mouseWorldPos = Camera.main.ScreenToWorldPoint(Input.mousePosition);
            Vector2 connectedAnchorWorldPos = anchor.position;// transform.TransformPoint(springJoint.connectedAnchor);
            Debug.Log(connectedAnchorWorldPos);
            Vector2 mouseDir = (mouseWorldPos - connectedAnchorWorldPos).normalized;
            float mouseLength = Mathf.Abs(Vector2.Distance(mouseWorldPos, connectedAnchorWorldPos));//springJoint.distance;//

            Vector2 movePoint = mouseLength <= MAX_DISTANCE ?
                                mouseWorldPos :
                                connectedAnchorWorldPos + mouseDir * MAX_DISTANCE;

            //Debug.DrawLine(connectedAnchorWorldPos, movePoint, Color.cyan, 1);

            rb.MovePosition(movePoint);
            rb.isKinematic = true;
        }

    }



    private void FixedUpdate()
    {
        RaycastHit2D rayHit;

        if (isMouseDown)
        {
            rayHit = Physics2D.Raycast(Camera.main.ScreenToWorldPoint(Input.mousePosition), Vector3.back);
            if (rayHit.collider != null)
            {
                if (rayHit.collider.CompareTag("Bird"))
                {
                    HoldBird = true;
                }
            }
        }
    }

    private IEnumerator Release(float time)
    {
        currentBird.Release();
        yield return new WaitForSeconds(time);
        springJoint.enabled = false;
        releasedBird = currentBird;

        yield return new WaitForSeconds(1f);
        if (index < birds.Length)
        {
            currentBird = birds[index++];
            currentBird.transform.position = anchor.position;
            rb = currentBird.RB;
            springJoint = currentBird.SpringJoint;
        }
    }

}


Next, we can to create a script for our bird and attach it to the game object. I named my script to be "BirdController". Also, we want to create a "Bird" tag for our bird.

In [ ]:
using System;
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UIElements;

using UnityEngine.SceneManagement;

public class BirdController : MonoBehaviour
{
    public Rigidbody2D RB { get; private set; }
    public SpringJoint2D SpringJoint { get; private set; }

    // Start is called before the first frame update
    void Awake()
    {
        RB = GetComponent<Rigidbody2D>();
        SpringJoint = GetComponent<SpringJoint2D>();
    }

    protected virtual void Start()
    {
        isFlying = false;
    }

    public void Release()
    {
        isFlying = true;
    }
}


Now, go back to our slingshot component, we want to attach our birdcontroller to the array of birds. Drag in the anchor location to the anchor option. Change max distance to 5 or some appropriate value, and release time to 0.2. These values are subject to change if needed.
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_5.png" alt="Drawing"/>

If everything set up correctly, we should be able to do this:
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_2.gif" alt="Drawing"/>


# Concept 2: Inheritance
---
#### What is Inheritance? 




## Activity


---
## Activity #1

---

**Problem:** 

##### **Solution**


---
## Activity #2

---

**Problem:**

##### **Solution**


 

# HOMEWORK
---

## HW #1

---

**Problem:** 

##### **Solution**

